In [ ]:
from qiskit_optimization import QuadraticProgram
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_algorithms.utils import algorithm_globals

In [375]:
mod = QuadraticProgram("Optimization_Problem")
print(mod.prettyprint())

Problem name: Optimization_Problem

Minimize
  0

Subject to
  No constraints

  No variables



In [376]:
N=5
Dist=2 # Distance to travel

tolerance = 2 # Tolerance in distance travelled

delta_v=1 # Rate of acceleration/deceleration set to 1
delta_v_sq=delta_v**2

vmax=150 # Max speed of a TGV in France (in m/s)

alpha=0.05 # Regenerative braking efficiency

# Define two lists of binary variables
x = mod.binary_var_list(N, name="x")
y = mod.binary_var_list(N, name="y")
print(mod.prettyprint())

Problem name: Optimization_Problem

Minimize
  0

Subject to
  No constraints

  Binary variables (10)
    x0 x1 x2 x3 x4 y5 y6 y7 y8 y9



### Defining objective function

In [377]:
linear_terms = {var.name: delta_v_sq for var in x}
linear_terms.update({var.name: -alpha*delta_v_sq for var in y})
mod.minimize(linear=linear_terms)
print(mod.prettyprint())

Problem name: Optimization_Problem

Minimize
  x0 + x1 + x2 + x3 + x4 - 0.05*y5 - 0.05*y6 - 0.05*y7 - 0.05*y8 - 0.05*y9

Subject to
  No constraints

  Binary variables (10)
    x0 x1 x2 x3 x4 y5 y6 y7 y8 y9



### Adding constraints

#### Net-Zero constraint

In [378]:
constraint_terms = {var.name: 1 for var in x}
constraint_terms.update({var.name: -1 for var in y})
mod.linear_constraint(constraint_terms, '=', 0, name="net-zero_constraint")
print(mod.prettyprint())

Problem name: Optimization_Problem

Minimize
  x0 + x1 + x2 + x3 + x4 - 0.05*y5 - 0.05*y6 - 0.05*y7 - 0.05*y8 - 0.05*y9

Subject to
  Linear constraints (1)
    x0 + x1 + x2 + x3 + x4 - y5 - y6 - y7 - y8 - y9 == 0  'net-zero_constraint'

  Binary variables (10)
    x0 x1 x2 x3 x4 y5 y6 y7 y8 y9



#### Maximum Speed Constraint

In [379]:
constraint_terms = {var.name: delta_v for var in x}
mod.linear_constraint(constraint_terms, '<=', vmax, name="Maximum_Speed_constraint")
print(mod.prettyprint())

Problem name: Optimization_Problem

Minimize
  x0 + x1 + x2 + x3 + x4 - 0.05*y5 - 0.05*y6 - 0.05*y7 - 0.05*y8 - 0.05*y9

Subject to
  Linear constraints (2)
    x0 + x1 + x2 + x3 + x4 - y5 - y6 - y7 - y8 - y9 == 0  'net-zero_constraint'
    x0 + x1 + x2 + x3 + x4 <= 150  'Maximum_Speed_constraint'

  Binary variables (10)
    x0 x1 x2 x3 x4 y5 y6 y7 y8 y9



#### Positive speed constraint

In [380]:
'''for i in range(N):
    constraint_terms = {var.name: 1 for var in x[0:i]}
    constraint_terms.update({var.name: -1 for var in y[0:i]})
    mod.linear_constraint(constraint_terms, '>=', 0, name="Positive_Speed"+str(i))

print(mod.prettyprint())'''

'for i in range(N):\n    constraint_terms = {var.name: 1 for var in x[0:i]}\n    constraint_terms.update({var.name: -1 for var in y[0:i]})\n    mod.linear_constraint(constraint_terms, \'>=\', 0, name="Positive_Speed"+str(i))\n\nprint(mod.prettyprint())'

#### No simultaneous acceleration/deceleration

In [381]:
z = mod.binary_var_list(N, name="z")

# Constraint 1: z[i] <= x[i]
for i in range(N):
    mod.linear_constraint({x[i].name: 1, z[i].name: -1}, '<=', 0, name=f"z_u_d_{i}")

# Constraint 2: z[i] <= y[i]
for i in range(N):
    mod.linear_constraint({y[i].name: 1, z[i].name: -1}, '<=', 0, name=f"z_p_d_{i}")

# Constraint 3: z[i] >= x[i] + y[i] - 1
for i in range(N):
    mod.linear_constraint({x[i].name: 1, y[i].name: 1, z[i].name: -1}, '>=', -1, name=f"z_u_p_d_{i}")

# Final Constraint: sum(z[i]) == 0 (No simultaneous braking or acceleration)
sum_constraint = {z[i].name: 1 for i in range(N)}
mod.linear_constraint(sum_constraint, '==', 0, name="No_simultaneous_braking_or_acceleration_constraint")

# Print the QuadraticProgram to see the problem
print(mod.prettyprint())


Problem name: Optimization_Problem

Minimize
  x0 + x1 + x2 + x3 + x4 - 0.05*y5 - 0.05*y6 - 0.05*y7 - 0.05*y8 - 0.05*y9

Subject to
  Linear constraints (18)
    x0 + x1 + x2 + x3 + x4 - y5 - y6 - y7 - y8 - y9 == 0  'net-zero_constraint'
    x0 + x1 + x2 + x3 + x4 <= 150  'Maximum_Speed_constraint'
    x0 - z10 <= 0  'z_u_d_0'
    x1 - z11 <= 0  'z_u_d_1'
    x2 - z12 <= 0  'z_u_d_2'
    x3 - z13 <= 0  'z_u_d_3'
    x4 - z14 <= 0  'z_u_d_4'
    y5 - z10 <= 0  'z_p_d_0'
    y6 - z11 <= 0  'z_p_d_1'
    y7 - z12 <= 0  'z_p_d_2'
    y8 - z13 <= 0  'z_p_d_3'
    y9 - z14 <= 0  'z_p_d_4'
    x0 + y5 - z10 >= -1  'z_u_p_d_0'
    x1 + y6 - z11 >= -1  'z_u_p_d_1'
    x2 + y7 - z12 >= -1  'z_u_p_d_2'
    x3 + y8 - z13 >= -1  'z_u_p_d_3'
    x4 + y9 - z14 >= -1  'z_u_p_d_4'
    z10 + z11 + z12 + z13 + z14
    == 0  'No_simultaneous_braking_or_acceleration_constraint'

  Binary variables (15)
    x0 x1 x2 x3 x4 y5 y6 y7 y8 y9 z10 z11 z12 z13 z14



In [382]:
# Define the linear expression for the total distance
linear_terms = {}
for i in range(N):
    # Each term in the summation contributes delta_v * (x[i] - y[i])
    linear_terms[x[i].name] = delta_v  # x[i] contributes +delta_v
    linear_terms[y[i].name] = -delta_v  # y[i] contributes -delta_v

# Add constraints for distance (distance <= Dist + tolerance) and (distance >= Dist - tolerance)
mod.linear_constraint(linear_terms, '<=', Dist + tolerance, name="Max_Distance_constraint")
mod.linear_constraint(linear_terms, '>=', Dist - tolerance, name="Min_Distance_constraint")

# Print the QuadraticProgram to see the problem
print(mod.prettyprint())

Problem name: Optimization_Problem

Minimize
  x0 + x1 + x2 + x3 + x4 - 0.05*y5 - 0.05*y6 - 0.05*y7 - 0.05*y8 - 0.05*y9

Subject to
  Linear constraints (20)
    x0 + x1 + x2 + x3 + x4 - y5 - y6 - y7 - y8 - y9 == 0  'net-zero_constraint'
    x0 + x1 + x2 + x3 + x4 <= 150  'Maximum_Speed_constraint'
    x0 - z10 <= 0  'z_u_d_0'
    x1 - z11 <= 0  'z_u_d_1'
    x2 - z12 <= 0  'z_u_d_2'
    x3 - z13 <= 0  'z_u_d_3'
    x4 - z14 <= 0  'z_u_d_4'
    y5 - z10 <= 0  'z_p_d_0'
    y6 - z11 <= 0  'z_p_d_1'
    y7 - z12 <= 0  'z_p_d_2'
    y8 - z13 <= 0  'z_p_d_3'
    y9 - z14 <= 0  'z_p_d_4'
    x0 + y5 - z10 >= -1  'z_u_p_d_0'
    x1 + y6 - z11 >= -1  'z_u_p_d_1'
    x2 + y7 - z12 >= -1  'z_u_p_d_2'
    x3 + y8 - z13 >= -1  'z_u_p_d_3'
    x4 + y9 - z14 >= -1  'z_u_p_d_4'
    z10 + z11 + z12 + z13 + z14
    == 0  'No_simultaneous_braking_or_acceleration_constraint'
    x0 + x1 + x2 + x3 + x4 - y5 - y6 - y7 - y8 - y9
    <= 4  'Max_Distance_constraint'
    x0 + x1 + x2 + x3 + x4 - y5 - y6 - y7 

### Convert optimization problem

In [383]:
qp2qubo = QuadraticProgramToQubo()
qubo = qp2qubo.convert(mod)
op, offset = qubo.to_ising()
print("offset: {}".format(offset))
print("operator:")
print(op)

offset: 888464.2500000005
operator:
SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZ', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZI', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIII

In [384]:
qp = QuadraticProgram()
qp.from_ising(op, offset, linear=True)
print(qp.prettyprint())

Problem name: 

Minimize
  1046*x0*x1 - 267.75*x0*x10 + 261.5*x0*x15 + 523*x0*x16 + 1046*x0*x17
  + 2092*x0*x18 + 4184*x0*x19 + 1046*x0*x2 + 8368*x0*x20 + 16736*x0*x21
  + 6014.5*x0*x22 - 261.5*x0*x23 - 523*x0*x24 + 1046*x0*x3 + 261.5*x0*x33
  + 523*x0*x34 + 1046*x0*x35 + 523*x0*x36 - 261.5*x0*x37 - 523*x0*x38
  - 523*x0*x39 + 1046*x0*x4 - 523*x0*x5 - 784.5*x0*x6 - 784.5*x0*x7
  - 784.5*x0*x8 - 784.5*x0*x9 - 267.75*x1*x11 + 261.5*x1*x15 + 523*x1*x16
  + 1046*x1*x17 + 2092*x1*x18 + 4184*x1*x19 + 1046*x1*x2 + 8368*x1*x20
  + 16736*x1*x21 + 6014.5*x1*x22 - 261.5*x1*x25 - 523*x1*x26 + 1046*x1*x3
  + 261.5*x1*x33 + 523*x1*x34 + 1046*x1*x35 + 523*x1*x36 - 261.5*x1*x37
  - 523*x1*x38 - 523*x1*x39 + 1046*x1*x4 - 784.5*x1*x5 - 523*x1*x6 - 784.5*x1*x7
  - 784.5*x1*x8 - 784.5*x1*x9 + 261.5*x10*x11 + 261.5*x10*x12 + 261.5*x10*x13
  + 261.5*x10*x14 + 261.5*x10*x23 + 523*x10*x24 + 261.5*x11*x12 + 261.5*x11*x13
  + 261.5*x11*x14 + 261.5*x11*x25 + 523*x11*x26 + 261.5*x12*x13 + 261.5*x12*x14
  + 261.5*

Our optimization problem is now defined, let's solve it!

### Get backend

In [385]:
#QiskitRuntimeService.save_account(channel="ibm_quantum", token="127d9370ab5b85e463bb2aa2196cb842ded206746763d1197bf58b29fa2a34eefcb8651384d8e1ca8e17301293ec53cf9c5dd55acbe12113630dbf575ce497df", set_as_default=True)
# Load saved credentials
service = QiskitRuntimeService()
# Get the least busy backend
backend = service.least_busy(operational=True, simulator=False)
print("least busy backend: ", backend)

least busy backend:  <IBMBackend('ibm_kyiv')>


In [386]:
algorithm_globals.random_seed = 10598
qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA())
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA

print(qaoa.is_compatible(problem=qp))
print(qaoa.get_compatibility_msg(problem=qp))




True



C:\Users\arthu\AppData\Local\Temp\ipykernel_16684\1423153282.py:2: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA())


In [388]:
qaoa_result = qaoa.solve(problem=qubo)
print(qaoa_result.prettyprint())

MemoryError: Unable to allocate 16.0 TiB for an array with shape (1099511627776,) and data type complex128

In [ ]:
sampler=Sampler(mode=backend)
optimizer=COBYLA(maxiter=100)
qaoa=QAOA(sampler=sampler, optimizer=optimizer)
solver = MinimumEigenOptimizer(qaoa)

In [ ]:
qaoa_result = qaoa.solve(qubo)
print(qaoa_result.prettyprint())

KeyboardInterrupt: 